In [109]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pd
from torch import utils as t_utils
from torch.utils.data.dataset import random_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import sys

from helpers import train_loop, test_loop
# device = torch.device("cuda" if torch.cuda.is_available() else "mps")
device = torch.device("cuda")
sys.dont_write_bytecode = True
torch.manual_seed(17)



In [2]:

train_data = pd.read_csv('./data/train.csv',dtype = np.float32)

train_data['label'] = train_data['label'].astype(int)
# test_data['label'] = test_data['label'].astype(int)

# convert to numpy
targets_numpy = train_data.label.values
features_numpy = train_data.loc[:,train_data.columns != "label"].values

# convert to tensor
targets_tensor = torch.from_numpy(targets_numpy).to(device).long()
features_tensor = torch.from_numpy(features_numpy).to(device).float()

# create dataset
dataset = TensorDataset(features_tensor, targets_tensor)
 
# split dataset
rows = train_data.shape[0]
test_size = int(rows*0.2)
train_size = rows - test_size
trainDataset, testDataset = random_split(dataset, [train_size, test_size])

# init loaders
batch_size = 100
learning_rate = 1e-3 
epochs = 5

trainLoader = DataLoader(trainDataset, batch_size=batch_size)
testLoader = DataLoader(testDataset, batch_size=batch_size)

In [3]:
from models.SimpleLinearNeuralNetwork import SimpleLinearNeuralNetwork
    
simpleLinearNeuralNetworkModel = SimpleLinearNeuralNetwork().to(device)
print(simpleLinearNeuralNetworkModel)

SimpleLinearNeuralNetwork(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [4]:


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(simpleLinearNeuralNetworkModel.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainLoader, simpleLinearNeuralNetworkModel, loss_fn, optimizer)

test_loop(testLoader, simpleLinearNeuralNetworkModel, loss_fn)

print("Done!")

Epoch 1
-------------------------------
loss: 72.123962  [  100/33600]
loss: 16.933544  [12900/33600]
loss: 16.350508  [25700/33600]
Epoch 2
-------------------------------
loss: 14.734322  [  100/33600]
loss: 11.801654  [12900/33600]
loss: 19.451086  [25700/33600]
Epoch 3
-------------------------------
loss: 10.727927  [  100/33600]
loss: 11.365089  [12900/33600]
loss: 5.524220  [25700/33600]
Epoch 4
-------------------------------
loss: 40.182823  [  100/33600]
loss: 12.726736  [12900/33600]
loss: 6.730944  [25700/33600]
Epoch 5
-------------------------------
loss: 10.690359  [  100/33600]
loss: 16.029102  [12900/33600]
loss: 8.447862  [25700/33600]
Test Error: 
 Accuracy: 80.4%, Avg loss: 22.483547 

Done!


In [5]:

loaderIterator = iter(testLoader) 

In [11]:

from models.ComplexLinearNeuralNetwork import ComplexLinearNeuralNetwork

 
        
complexLinearNeuralNetwork = ComplexLinearNeuralNetwork().to(device)
print(complexLinearNeuralNetwork)

ComplexLinearNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=150, bias=True)
    (1): ReLU()
    (2): Linear(in_features=150, out_features=150, bias=True)
    (3): Tanh()
    (4): Linear(in_features=150, out_features=150, bias=True)
    (5): ELU(alpha=1.0)
    (6): Linear(in_features=150, out_features=10, bias=True)
  )
)


In [12]:
from IPython.display import clear_output


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(complexLinearNeuralNetwork.parameters(), lr=0.01)
epochs = 30
for t in range(epochs):
    clear_output(wait=False)
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainLoader, complexLinearNeuralNetwork, loss_fn, optimizer)
     
    # test_loop(testLoader, complexLinearNeuralNetwork, loss_fn)

Epoch 30
-------------------------------
loss: 0.006919  [  100/33600]
loss: 0.005704  [12900/33600]
loss: 0.003312  [25700/33600]


In [23]:
test_loop(testLoader, complexLinearNeuralNetwork, loss_fn)

Test Error: 
 Accuracy: 96.6%, Avg loss: 0.122865 



In [111]:
class ConvNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__() 
        # self.linear_relu_stack = nn.Sequential(
        #     nn.Conv2d(kernel_size=3, stride=1, padding=0, in_channels=1, out_channels=16),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2),
        #     nn.Conv2d(kernel_size=3, stride=1, padding=2, in_channels=16, out_channels=8),
        #     nn.ReLU(),
        #     nn.Conv2d(kernel_size=3, stride=1, padding=2, in_channels=8, out_channels=4),
        #     nn.ReLU(),
        #     nn.Flatten(),
        #     nn.Linear(1156, 10),

        # )

        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(kernel_size=3, stride=1, padding=0, in_channels=1, out_channels=20),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.Flatten(),
            nn.Linear(3380, 10),

        )

        self.conv1 = nn.Conv2d(kernel_size=3, stride=1, padding=0, in_channels=1, out_channels=16)
        self.relu1 = nn.ReLU()
        self.maxPool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(kernel_size=3, stride=1, padding=0, in_channels=1, out_channels=8)
        self.relu2 = nn.ReLU()
        self.maxPool2 = nn.MaxPool2d(kernel_size=2, stride=2) 
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(1352, 10)

    def forward(self, x):
        return self.linear_relu_stack(x)
        out = self.conv1(x)
        print(f"conv1 out: {out.size()}")
        out = self.relu1(out)
        print(f"relu1 out: {out.size()}")
        out = self.maxPool1(out)
        print(f"maxPool1 out: {out.size()}") 
        out = self.conv2(x)
        print(f"conv2 out: {out.size()}")
        out = self.relu2(out)
        print(f"relu2 out: {out.size()}")
   
        out = self.maxPool2(out)
        print(f"maxPool1 out: {out.size()}") 

        out = self.flatten(out)
        print(f"flatten out: {out.size()}")

        out = self.linear1(out)
        print(f"linear1 out: {out.size()}") 
        # self.linear_relu_stack(x)
        return out
    
convNeuralNetwork = ConvNeuralNetwork().to(device)
print(convNeuralNetwork)

ConvNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Flatten(start_dim=1, end_dim=-1)
    (4): Linear(in_features=2704, out_features=10, bias=True)
  )
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
  (relu1): ReLU()
  (maxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (relu2): ReLU()
  (maxPool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=1352, out_features=10, bias=True)
)


In [112]:
from IPython.display import clear_output

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(convNeuralNetwork.parameters(), lr=0.001)
epochs = 5

def reshape(x):
    return x.view(batch_size,1,28,28)

for t in range(epochs):
    clear_output(wait=False)
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainLoader, convNeuralNetwork, loss_fn, optimizer, reshape)
     
    # test_loop(testLoader, complexLinearNeuralNetwork, loss_fn)

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (100x3380 and 2704x10)

In [107]:
# def test_loop(dataloader, model, loss_fn):
#     size = len(dataloader.dataset)
#     num_batches = len(dataloader)
#     correct = 0
#     test_loss = 0 
#     with torch.no_grad():
#         for (X, y) in dataloader: 
#             pred = model(X.view(100,1,28,28)) 
#             test_loss += loss_fn(pred, y).item()
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item() 
 
#     test_loss /= num_batches
#     correct /= size
#     print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [110]:
test_loop(testLoader, convNeuralNetwork, loss_fn, reshape)

Test Error: 
 Accuracy: 96.7%, Avg loss: 0.118092 



In [69]:

test_data = pd.read_csv('./data/test.csv',dtype = np.float32)
 
test_features_tensor = torch.from_numpy(test_data.values).to(device).float()

class MyDataset(Dataset):
    def __init__(self):
        self.data = torch.from_numpy(test_data.values).to(device).float()
        
    def __getitem__(self, index):
        x = self.data[index]
        return x
    
    def __len__(self):
        return len(self.data)
 
test_dataset = MyDataset()
test_Loader = DataLoader(test_dataset, batch_size=batch_size)

In [71]:
len(test_Loader.dataset)

28000

In [79]:
result = np.array([])
with torch.no_grad():
    for X in test_Loader:
        pred = convNeuralNetwork(X.view(100,1,28,28)) 
        result = np.append(result, pred.argmax(1).cpu().numpy()) 
         

In [101]:
dsd = pd.DataFrame(data={'ImageId': np.arange(1, len(result)+1), 'Label': result.astype(int)})

In [102]:
dsd.to_csv('submission.csv', index=False)